In [1]:
import lithops
import laspy.copc as lasp
import time
import pandas as pd
import numpy as np

In [2]:
from cloudAPI import CopcHelper

In [3]:
bucket= 'lithops-testing3'
filekey= 'autzen-classified.copc.laz'
filename= 'lidar.copc.laz'

In [4]:
def helper_test(aux, ibm_cos):
    clock0 = time.time()
    
    key = lasp.VoxelKey()
    key.x = 1
    key.y = 2
    key.z = 0
    key.level = 2
    
    #Download header and octree
    helper = CopcHelper(bucket, filekey, filename, ibm_cos)
    helper.load_header()
    helper.load_rootpage()
    
    clock1 = time.time()
    
    #Download and read points of a level 1 node
    helper.load_points(key)
    points = helper.get_points(key)
    
    clock2 = time.time()
    
    #Find neighbours of a level 1 node
    neighbours = helper.get_smallest_neighbours(key)
    
    clock3 = time.time()
    
    #Download and load points of neighbours
    neighbour_points = []
    for neighbour in neighbours:
        helper.load_points(neighbour)
        neighbour_points.append(helper.get_points(neighbour))
    
    clock4 = time.time()

    return clock1 - clock0, clock2 - clock1, clock3 - clock2, clock4 - clock3, key

In [5]:
if __name__ == "__main__":
    fexec = lithops.LocalhostExecutor()
    fexec.call_async(helper_test, 0)
    results = fexec.get_result()

2022-07-13 09:19:54,430 [INFO] lithops.config -- Lithops v2.6.0
2022-07-13 09:19:54,433 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-07-13 09:19:54,433 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-07-13 09:19:54,434 [INFO] lithops.invokers -- ExecutorID 1074e6-0 | JobID A000 - Selected Runtime: python 
2022-07-13 09:19:54,517 [INFO] lithops.invokers -- ExecutorID 1074e6-0 | JobID A000 - Starting function invocation: helper_test() - Total: 1 activations
2022-07-13 09:19:54,550 [INFO] lithops.invokers -- ExecutorID 1074e6-0 | JobID A000 - View execution logs at /tmp/lithops/logs/1074e6-0-A000.log
2022-07-13 09:19:55,400 [INFO] lithops.wait -- ExecutorID 1074e6-0 - Getting results from 1 function activations


    0%|          | 0/1  

2022-07-13 09:20:03,591 [INFO] lithops.executors -- ExecutorID 1074e6-0 - Cleaning temporary data


In [6]:
    print("EXECUTION TIMES OF KEY ", results[4])
    print("Download header and octree: ", results[0])
    print("Download and read points of a level 1 node: ", results[1])
    print("Find neighbours of a level 1 node: ", results[2])
    print("Download and read points of neighbours: ", results[3])
    print("Total time: ", results[0] + results[1] + results[2] + results[3])

EXECUTION TIMES OF KEY  VoxelKey(level=2, x=1, y=2, z=0)
Download header and octree:  0.7880523204803467
Download and read points of a level 1 node:  0.5349459648132324
Find neighbours of a level 1 node:  0.005063056945800781
Download and read points of neighbours:  5.575808525085449
Total time:  6.903869867324829
